In [1]:
import numpy as np
import pandas as pd
import time
import pickle

# For plotting

%matplotlib inline
import random
import networkx
#UMAP
import umap.umap_ as umap
import shap
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr

from sklearn.cluster import OPTICS
import numpy as np


import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from matplotlib import rcParams
import os
import networkx
import itertools


In [2]:
picklepath = 'C:\\Users\\qdickinson\\Jupyter Notebooks\\yeast_multiomics_impute_SHAP\\pickles\\'
pro_all = pd.read_pickle(picklepath+'pro_all.pickle')
met_all = pd.read_pickle(picklepath+'met_all.pickle')
met_shap = pickle.load( open( picklepath + 'shap_pmet_all.pickle', "rb" ))
shap_values = np.asarray(list(met_shap.values()))


# UMAP

In [ ]:
#Create Example UMAP figure

rcParams['figure.figsize'] = 13,10
start = time.time()
y = list(pro_all.index)
x = shap_values[20]

reducer = umap.UMAP(n_components=2,random_state = 42, n_neighbors=4, min_dist = 1, metric='manhattan')
embedding =  reducer.fit_transform(x)

pcx=reducer.embedding_[:, 0]
pcy=reducer.embedding_[:, 1]

i=0


condition = []
for value in list(pro_all.index):
    condition.append(value.split("_")[1])

clustering = OPTICS(min_samples=2).fit(reducer.embedding_)
#sns.scatterplot(x=pcx, y=pcy, hue = colorclusters,
#                style = condition, palette = sns.color_palette("Set2", 100, as_cmap = True), s=200)
#sns.scatterplot(x=x_ribo, y=y_ribo,
#                 palette = sns.color_palette("gray"), s=1)

norm = plt.Normalize(np.array(clustering.labels_).min(), np.array(clustering.labels_).max())
#norm = plt.Normalize(np.array(colorclusters).min(), np.array(colorclusters).max())
sm = plt.cm.ScalarMappable(cmap=sns.color_palette("Spectral_r", 30, as_cmap = True), norm=norm)
sm.set_array([])




kwargs  =   {'edgecolor':"black", # for edge color
             'linewidth':1, # line width of spot
            }

ax = sns.scatterplot(x=pcx, y=pcy, hue = clustering.labels_,#colorclusters,
                style = condition, palette = sns.color_palette("Spectral_r", 30, as_cmap = True), s=200,
                **kwargs)
#ax.get_legend().remove()
ax.figure.colorbar(sm)
plt.savefig('YJR120W_umap_clusterNOumap.svg')
plt.show()

In [3]:
#compound loop of umaploop

y = list(pro_all.index)
compound = 0
compoundclustersarray = []
while compound < len(list(met_shap.keys())):
    if compound == 65:
        compound = 265
    x = shap_values[compound]
    #x = pro_all.values
    iterator = 0
    repeats = 1000
    labelarray = []
    nclusters = []
    #x = StandardScaler().fit_transform(x)

    while iterator<repeats:
        reducer = umap.UMAP(n_components=10, n_neighbors=3, min_dist = 0, metric='manhattan')
        embedding =  reducer.fit_transform(x)
        clustering = OPTICS(min_samples=2).fit(reducer.embedding_)
        labelarray.append(clustering.labels_)
        iterator+=1

    clustersarray = []

    i = 0
    while i < len(labelarray):
        j=0
        temp = []
        while j < max(labelarray[i])+1:
            temp.append([])
            j+=1
        j = 0
        while j < len(labelarray[i]):
            if labelarray[i][j] != -1:
                temp[int(labelarray[i][j])].append(y[j])
            j+=1
        for value in temp:
            clustersarray.append(value)
        i+=1
    #compoundclustersarray.append(clustersarray)

    
    pickle.dump( clustersarray, open(picklepath + "clusters\\test\\" + str(compound)+ "-" + str(list(met_shap.keys())[compound].split(" ")[0]) + ".pickle", "wb" ))
    compound+=1
    print(str(compound)+ "-" + str(list(met_shap.keys())[compound].split(" ")[0]))

1-2-Aminoheptanedioic
2-2-Hydroxyglutaric


KeyboardInterrupt: 

In [4]:
#Create Network with all conditions and connections across metabolites



#Get a list of all clusters from cluster pickles to get cluster overlaps
#create graph containing all connections
files = []
allclusters = []
#allfinalclusters=[]

clusterpath = picklepath + "clusters\\"
files = []
for file in os.listdir(clusterpath):
    path = os.path.join(clusterpath, file)
    if os.path.isdir(path):
        # skip directories
        continue
    else:
        files.append(file)

for value in files:
    allclusters.append(pickle.load( open( clusterpath + value, "rb" )))
G = networkx.Graph()    
compound = 0
while compound < len(allclusters):
    

    i = 0
    while i < len(allclusters[compound]):
        j = 0
        if compound == 0:
            while j < len(allclusters[compound][i]):
                G.add_node(allclusters[compound][i][j])
                j+=1
        j = 0
        while j < len(allclusters[compound][i]):
            k = 0
            while k < len(allclusters[compound][i]):
                if G.has_edge(allclusters[compound][i][j], allclusters[compound][i][k]):
                    # we added this one before, just increase the weight by one
                    G[allclusters[compound][i][j]][allclusters[compound][i][k]]['weight'] += 1
                else:
                    # new edge. add with weight=1
                    G.add_edge(allclusters[compound][i][j], allclusters[compound][i][k], weight=1)
                k+=1
            j+=1
        i+=1
        
    G.remove_edges_from(networkx.selfloop_edges(G))


    compound+=1
    print(compound/len(allclusters))
#pickle.dump(allfinalclusters, open("allfinalclusters_new.pickle", 'wb'))
networkx.write_graphml(G,"allconnectionsallmetabolites.xml")

0.014492753623188406
0.028985507246376812
0.043478260869565216
0.057971014492753624
0.07246376811594203
0.08695652173913043
0.10144927536231885
0.11594202898550725
0.13043478260869565
0.14492753623188406
0.15942028985507245
0.17391304347826086
0.18840579710144928
0.2028985507246377
0.21739130434782608
0.2318840579710145
0.2463768115942029
0.2608695652173913
0.2753623188405797
0.2898550724637681
0.30434782608695654
0.3188405797101449
0.3333333333333333
0.34782608695652173
0.36231884057971014
0.37681159420289856
0.391304347826087
0.4057971014492754
0.42028985507246375
0.43478260869565216
0.4492753623188406
0.463768115942029
0.4782608695652174
0.4927536231884058
0.5072463768115942
0.5217391304347826
0.5362318840579711
0.5507246376811594
0.5652173913043478
0.5797101449275363
0.5942028985507246
0.6086956521739131
0.6231884057971014
0.6376811594202898
0.6521739130434783
0.6666666666666666
0.6811594202898551
0.6956521739130435
0.7101449275362319
0.7246376811594203
0.7391304347826086
0.7536231

In [5]:
#create networks for each metabolites and output weights in csv

nodes_list = list(G.nodes)
combined_nodes_list = itertools.combinations(nodes_list, 2)
combined_nodes_list = list(combined_nodes_list)

edges_list = []

for value in combined_nodes_list:
    edges_list.append(str(value[0]) + " (-) " + str(value[1]))
    
weightsdf = pd.DataFrame()
weightsdf['Edges'] = edges_list
#Get a list of all clusters from cluster pickles to get cluster overlaps
#create graph containing all connections
files = []
allclusters = []
#allfinalclusters=[]

clusterpath = picklepath + "clusters\\"
files = []
for file in os.listdir(clusterpath):
    path = os.path.join(clusterpath, file)
    if os.path.isdir(path):
        # skip directories
        continue
    else:
        files.append(file)

for value in files:
    allclusters.append(pickle.load( open( clusterpath + value, "rb" )))
G = networkx.Graph()    
compound = 0
while compound < len(allclusters):
    Gtemp = networkx.Graph()  

    i = 0
    while i < len(allclusters[compound]):
        j = 0
        if compound == 0:
            while j < len(allclusters[compound][i]):
                G.add_node(allclusters[compound][i][j])
                Gtemp.add_node(allclusters[compound][i][j])
                j+=1
        j = 0
        while j < len(allclusters[compound][i]):
            k = 0
            while k < len(allclusters[compound][i]):
                if G.has_edge(allclusters[compound][i][j], allclusters[compound][i][k]):
                    # we added this one before, just increase the weight by one
                    G[allclusters[compound][i][j]][allclusters[compound][i][k]]['weight'] += 1
                else:
                    # new edge. add with weight=1
                    G.add_edge(allclusters[compound][i][j], allclusters[compound][i][k], weight=1)
                    
                if Gtemp.has_edge(allclusters[compound][i][j], allclusters[compound][i][k]):
                    # we added this one before, just increase the weight by one
                    Gtemp[allclusters[compound][i][j]][allclusters[compound][i][k]]['weight'] += 1
                else:
                    # new edge. add with weight=1
                    Gtemp.add_edge(allclusters[compound][i][j], allclusters[compound][i][k], weight=1)
                k+=1
            j+=1
        i+=1
        
    G.remove_edges_from(networkx.selfloop_edges(G))
    Gtemp.remove_edges_from(networkx.selfloop_edges(G))
    networkx.write_graphml(Gtemp,str(files[compound].split(".")[0])+"_clusterconnections.xml")
    
    #add data to dataframe
    temp_weight_list = []
    for value in edges_list:
        temp_weight_list.append(0)
    
    graph_edges = []
    for value in list(Gtemp.edges.data()):
        temp_edge_str1 = str(value[0]) + " (-) " + str(value[1])
        temp_edge_str2 = str(value[1]) + " (-) " + str(value[0])
        
        if temp_edge_str1 in edges_list:
            temp_weight_list[edges_list.index(temp_edge_str1)] = value[2]['weight']
        elif temp_edge_str2 in edges_list:
            temp_weight_list[edges_list.index(temp_edge_str2)] = value[2]['weight']
        elif temp_edge_str2 == temp_edge_str1:
            a = 0
        else:
        
            print("not found")
            print(temp_edge_str1)
            print(temp_edge_str2)
    
    weightsdf[str(files[compound].split(".")[0])] = temp_weight_list
    compound+=1
    print(compound/len(allclusters))
#pickle.dump(allfinalclusters, open("allfinalclusters_new.pickle", 'wb'))
weightsdf.to_csv("weightsdf_test.csv")

0.014492753623188406
0.028985507246376812
0.043478260869565216
0.057971014492753624
0.07246376811594203
0.08695652173913043
0.10144927536231885
0.11594202898550725
0.13043478260869565
0.14492753623188406
0.15942028985507245
0.17391304347826086
0.18840579710144928
0.2028985507246377
0.21739130434782608
0.2318840579710145
0.2463768115942029
0.2608695652173913
0.2753623188405797
0.2898550724637681
0.30434782608695654
0.3188405797101449
0.3333333333333333
0.34782608695652173
0.36231884057971014
0.37681159420289856
0.391304347826087
0.4057971014492754
0.42028985507246375
0.43478260869565216
0.4492753623188406
0.463768115942029
0.4782608695652174
0.4927536231884058
0.5072463768115942
0.5217391304347826
0.5362318840579711
